LABORATORIO 1 - INTELIGENCIA ARTIFICIAL
Diego García 22404
César López 22838

# TASK 1  
 ### - PREGUNTA 1
Considera un modelo de regresión lineal con dos características, X₁ y X₂, y sus pesos correspondientes w₁ y w₂. 
Si el modelo predice una salida y mediante la ecuación y = 2w₁X₁ + 3w₂X₂ + 1, ¿cuál es la interpretación del coeficiente 3w₂ en el contexto del modelo?

- RESPUESTA:
3w₂ representa el cambio que recibe y cuando X₂ va cambiando sus valores. Esto significa que mientras mayor sea el valor de 3w₂ mayor será el valor final de y con cada valor distinto de X₂.


### - PREGUNTA 2  
Explica el concepto de multicolinealidad en el contexto de la regresión lineal. ¿Cómo afecta la
multicolinealidad a la interpretación de los coeficientes de regresión individuales?

- RESPUESTA:
Es cuando dos o mas variables predictoras estan altamente correlacionadas entre si, lo que implica que existe una redundancia en la informacion analizada, de donde se obtienen estas variables. Esto puede llegar a afectar aumentando el cesgo de las predicciones que realiza el modelo fuera del contexto con el que fue entrenado. Tambien el modelo puede llegar a confundir en cual es la variable que influye en la otra.

TASK 2 - CLASIFICACIÓN DE SITIOS DE PHISHING

Descargar el dataset de: https://www.kaggle.com/datasets/shashwatwork/web-page-phishing-detection-dataset 


In [ ]:
#Librerías para cargar los datos

import pandas as pd
import numpy as np

In [ ]:
dataset = 